In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum emission

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from sumo_grid_simulation.grid_simulation import Simulator
from experimental_design import config

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        beta = 0.05
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * beta)
        
        s = simulator.simulate(
            junctionType  = 2,
            tlType        = 2, 
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        
        num_veh = max_number_of_vehicles * beta
        CO2 = s['CO2'] / (num_veh * s['duration'])
        result.append(CO2)
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
#init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/270_init_points_emissions.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)
num_init_points = 520
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization/init_points/{num_init_points}_init_points_emissions.pkl', "wb") as f:
     pickle.dump(init_points, f)
'''

'\ndesign = RandomDesign(parameter_space)\nnum_init_points = 520\ninit_X = design.get_samples(num_init_points)\ninit_Y = user_function_emissions(init_X)\ninit_points = init_X, init_Y\nwith open(f\'bayesian_optimization/init_points/{num_init_points}_init_points_emissions.pkl\', "wb") as f:\n     pickle.dump(init_points, f)\n'

To then fit a GP to these points

In [7]:
#emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
#emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
#emukit_model_random_init.optimize()

### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [8]:
#emukit_model_variance = pickle.load(open('experimental_design/models/ms_5param_model_variance_20_init_points_500_loops_co2_per_duration.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_integrated_variance = pickle.load(open('experimental_design/models/ARD_5param_integrated_variance_reduction_250_init_points_500_loops_co2_per_duration.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 50

### Acquisition functions

In [11]:
#acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
#acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [13]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [14]:
#bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
#bo_random_init.run_loop(user_function_emissions, n_iter_bo)

In [15]:
#bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
#bo_model_variance.run_loop(user_function_emissions, n_iter_bo)

In [16]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_emissions, n_iter_bo)

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 3193.9766401924157
Optimization restart 2/5, f = 3717.5185345453606
Optimization restart 3/5, f = 3719.4583820216403
Optimization restart 4/5, f = 3714.8076291606303
Optimization restart 5/5, f = 3707.8686074072302
[[20.         8.        70.         1.         3.2477963]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1769.2654017907528



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 3200.1806379947075
Optimization restart 2/5, f = 74117.58478390695


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add


...

Optimization restart 3/5, f = 4066.144278912484
Optimization restart 4/5, f = 3966.2255476147525
Optimization restart 5/5, f = 4046.6507996953615
[[20.   8.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1816.4494689191501



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 3466.516999628122
Optimization restart 2/5, f = 4021.807583165562
Optimization restart 3/5, f = 4124.846577911028
Optimization restart 4/5, f = 4046.1209581111266


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = 4133.855739314754


## Results

In [17]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [18]:
#results_random_init = bo_random_init.get_results()

In [19]:
'''
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df
'''

'\nresults_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))\nresults_random_init_df.columns = parameters_name\nresults_random_init_df\n'

In [20]:
#results_random_init.minimum_value

### Results on the experimentally designed emulator with model variance

In [21]:
#results_model_variance = bo_model_variance.get_results()

In [22]:
'''
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df
'''

'\nresults_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))\nresults_random_init_df.columns = parameters_name\nresults_random_init_df\n'

In [23]:
#results_model_variance.minimum_value

### Results on the experimentally designed emulator with integrated variance reduction

In [24]:
results_integrated_variance = bo_integrated_variance.get_results()

In [25]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,56.729611,1.0,2.846833


In [26]:
results_integrated_variance.minimum_value

1730.7915117780626

## Save stuff

In [27]:
#with open(f'bayesian_optimization/bayesian_opt_results/results_random_init_emissions.pkl', "wb") as f:
#     pickle.dump(results_random_init, f)

#with open(f'bayesian_optimization/bayesian_opt_results/results_model_variance_emissions.pkl', "wb") as f:
#    pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization/bayesian_opt_results/results_integrated_variance_emissions_ARD.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [28]:
#with open(f'bayesian_optimization/models/520_random_init_50_bay_opt_emissions.pkl', "wb") as f:
#     pickle.dump(emukit_model_random_init, f)

#with open(f'bayesian_optimization/models/520_model_variance_50_bay_emissions.pkl', "wb") as f:
#     pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization/models/520_integrated_variance_50_bay_emissions_ARD.pkl', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)